# Analyzing and Formatting Sales Data with SQL

In this project, I worked with data from a multinational retail corporation. I used SQL to clean and analyze their data with a goal of identifying the businesses top categories based on the highest profit margins as well as detecting any missing values in their data.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |


In [8]:
-- Find top 5 products from each category based on highest total sales
SELECT * FROM (
SELECT products.category, 
	products.product_name,
  	ROUND(SUM(CAST(ord.sales AS NUMERIC)), 2) AS product_total_sales,
	ROUND(SUM(CAST(ord.profit AS NUMERIC)), 2) AS product_total_profit,
	RANK() OVER(PARTITION BY products.category ORDER BY SUM(ord.sales) DESC) AS product_rank
	FROM orders AS ord
	INNER JOIN products
		ON ord.product_id = products.product_id
	GROUP BY products.category, products.product_name
) AS tmp
WHERE product_rank < 6;

,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5


In [9]:
-- impute missing values
WITH missing AS (
	SELECT product_id,
		discount, 
		market,
		region,
		sales,
		quantity
	FROM orders 
	WHERE quantity IS NULL
), 

unit_prices AS (SELECT o.product_id,
	CAST(o.sales / o.quantity AS NUMERIC) AS unit_price
FROM orders o
RIGHT JOIN missing AS m 
	ON o.product_id = m.product_id
	AND o.discount = m.discount
WHERE o.quantity IS NOT NULL
)

SELECT DISTINCT m.*,
	ROUND(CAST(m.sales AS NUMERIC) / up.unit_price,0) AS calculated_quantity
FROM missing AS m
INNER JOIN unit_prices AS up
	ON m.product_id = up.product_id;

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4
1,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,2
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3
3,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2
4,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,4
